In [ ]:
"""
Created on Mon May 30 15:53:43 2022

@author: a2326
"""

In [ ]:
def full_frame(width=None, height=None):
    import matplotlib as mpl
    mpl.rcParams['savefig.pad_inches'] = 0
    figsize = None if width is None else (width, height)
    fig = plt.figure(figsize=figsize)
    ax = plt.axes([0,0,1,1], frameon=False)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    plt.autoscale(tight=True)

In [ ]:
from sklearn.cluster import SpectralClustering
import matplotlib.pyplot as plt
import numpy as np
import h5py
import sklearn.decomposition
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
#from fcmeans import FCM
#import skfuzzy as fuzz
from sklearn.manifold import TSNE
import collections
from numpy.ma import masked_array
from sklearn import mixture

In [ ]:
def solve(num):
   res = ""
   table = [
      (1000, "M"),
      (900, "CM"),
      (500, "D"),
      (400, "CD"),
      (100, "C"),
      (90, "XC"),
      (50, "L"),
      (40, "XL"),
      (10, "X"),
      (9, "IX"),
      (5, "V"),
      (4, "IV"),
      (1, "I"),
   ]
   for cap, roman in table:
      d, m = divmod(num, cap)
      res += roman * d
      num = m

   return res

#loading data##

In [ ]:
##unfolding data##
def unfold(data):
    data_unfold=data.reshape(data.shape[0]*data.shape[1], data.shape[2])
    return(data_unfold)
##folding data##
def fold(data):
    data_fold=data.reshape(n,n, data_unfold.shape[1])
    return(data_fold)
##dark counts##shouldnt this happen after the alignment
def df(data):
 darkcounts=np.zeros((data.shape[2]))
 for i in range(data.shape[2]):
      darkcounts[i]=(data[0:20,0:20,i].mean()+data[0:20,n-20:n,i].mean()+data[n-20:n,0:20,i].mean()+data[n-20:n,n-20:n,i].mean())/4
      data[:,:,i]=data[:,:,i]-darkcounts[i]
      data[:,:,i]=np.where(data[:,:,i]<0, 0, data[:,:,i])
 return(data)

In [ ]:
##io normalization##
def ionormalize(data,io):
    data_norm=(data*np.mean(io))/io
    return(data_norm)
#normalization to bright field image
def bfnormalize(data,norm):
    data_bf=(data/norm[:,:,None])*np.mean(norm)
    return(data_bf)

In [ ]:
#transform arrays#
def transformd(data):
    data=np.transpose(data,(1,0,2))
    datanew=np.flipud(data)
    return(datanew)
def transform(data):
    data=np.transpose(data)
    datanew=np.flipud(data)
    return(datanew)
def transform1(data):
    datanew=np.transpose(data,(1,0,2))
    return(datanew)
def transform2(data):
    datanew=np.flipud(data)
    return(datanew)
def transform4(data):
    datanew=np.transpose(np.flipud(data),(1,2,0))
    return(datanew)

In [ ]:
def uni(records_array):
    '''
    get all indices of each unique value in array
    '''
    idx_sort = np.argsort(records_array)
    sorted_records_array = records_array[idx_sort]
    vals, idx_start, count = np.unique(sorted_records_array, return_counts=True, return_index=True)
    res = np.split(idx_sort, idx_start[1:])
    return dict(zip(vals, res))

In [ ]:
def fov(data, resx, resy, vmin, vmax):
 font={'fontname':'Century Gothic'}
 plt.imshow(data, cmap='gray', extent=[0,data.shape[1]*resx,0,data.shape[0]*resy], vmin=vmin, vmax=vmax)
 plt.minorticks_on()
 plt.xlabel('x (μm)',weight="bold",**font)
 plt.ylabel('y (μm)',weight="bold",**font)

In [ ]:
def scale(data, resx, resy,cmap):
 font={'fontname':'Century Gothic'}
 plt.imshow(data, cmap=cmap, extent=[0,data.shape[1]*resx,0,data.shape[0]*resy])
 #plt.xticks([])
 #plt.yticks([])
 plt.minorticks_on()
 plt.xlabel('x (μm)',weight="bold",**font)
 plt.ylabel('y (μm)',weight="bold",**font)
 plt.colorbar(format='%.1f',fraction=0.046, pad=0.04)#.set_label('Optical Density', rotation=90,weight="bold", labelpad=20)

In [ ]:
def noscale(data, resx, resy,cmap):
  font={'fontname':'Century Gothic'}
  plt.imshow(data, cmap=cmap, extent=[0,data.shape[1]*resx,0,data.shape[0]*resy])
  plt.xticks([])
  plt.yticks([])

In [ ]:
def alignment(data):
 shift =np.zeros((2,data.shape[2]))
 error= np.zeros((data.shape[2]))
 diffphase=np.zeros((data.shape[2]))
 im_Cm=np.zeros((data.shape[0], data.shape[1], data.shape[2]))

 # align images with subpixel precision
 for i in range(data.shape[2]):
     shift[:,i], error[i], diffphase[i] = phase_cross_correlation(data[:,:,0], data[:,:,i], upsample_factor=20)#20 is 0.05 pixels
     im_Cm[:,:,i] = np.real( np.fft.ifftn(fourier_shift(np.fft.fftn(data[:,:,i]), shift[:,i])) )  
     #print(f'Detected pixel offset (y, x): {shift[:,i]}')

 data_aligned=np.zeros((data.shape[0]-round(max(shift[0,:]))+round(min(shift[0,:])), data.shape[1]-round(max(shift[1,:]))+round(min(shift[1,:])), data.shape[2]))
 for i in range(data.shape[2]):
    # data_aligned[:,:,i]=im_Cm[round(max(shift[0,:])):round(min(shift[0,:])), round(max(shift[1,:])):+round(min(shift[1,:])),i]
    if  round(min(shift[0,:]))!=0 and round(min(shift[1,:]))!=0:
        #print('case 1')
        data_aligned[:,:,i]=im_Cm[round(max(shift[0,:])):round(min(shift[0,:])), round(max(shift[1,:])):round(min(shift[1,:])),i]
    elif  round(min(shift[0,:]))==0 and round(min(shift[1,:]))!=0:
        #print('case 2')
        data_aligned[:,:,i]=im_Cm[round(max(shift[0,:])):, round(max(shift[1,:])):round(min(shift[1,:])),i]
    elif  round(min(shift[0,:]))!=0 and round(min(shift[1,:]))==0:
        #print('case 3')
        data_aligned[:,:,i]=im_Cm[round(max(shift[0,:])):round(min(shift[0,:])), round(max(shift[1,:])):,i]
    else:
        #print('case 4')
        data_aligned[:,:,i]=im_Cm[round(max(shift[0,:])):, round(max(shift[1,:])):,i]
 return shift, data_aligned

In [ ]:
def heteroalignment(data,shift):
 im_Cm=np.zeros((data.shape[0], data.shape[1], data.shape[2]))

 for i in range(data.shape[2]):
     im_Cm[:,:,i] = np.real( np.fft.ifftn(fourier_shift(np.fft.fftn(data[:,:,i]), shift[:,i])) )  
     #print(f'Detected pixel offset (y, x): {shift[:,i]}')

 data_aligned=np.zeros((data.shape[0]-round(max(shift[0,:]))+round(min(shift[0,:])), data.shape[1]-round(max(shift[1,:]))+round(min(shift[1,:])), data.shape[2]))
 for i in range(data.shape[2]):
    # data_aligned[:,:,i]=im_Cm[round(max(shift[0,:])):round(min(shift[0,:])), round(max(shift[1,:])):+round(min(shift[1,:])),i]
    if  round(min(shift[0,:]))!=0 and round(min(shift[1,:]))!=0:
        #print('case 1')
        data_aligned[:,:,i]=im_Cm[round(max(shift[0,:])):round(min(shift[0,:])), round(max(shift[1,:])):round(min(shift[1,:])),i]
    elif  round(min(shift[0,:]))==0 and round(min(shift[1,:]))!=0:
        #print('case 2')
        data_aligned[:,:,i]=im_Cm[round(max(shift[0,:])):, round(max(shift[1,:])):round(min(shift[1,:])),i]
    elif  round(min(shift[0,:]))!=0 and round(min(shift[1,:]))==0:
        #print('case 3')
        data_aligned[:,:,i]=im_Cm[round(max(shift[0,:])):round(min(shift[0,:])), round(max(shift[1,:])):,i]
    else:
        #print('case 4')
        data_aligned[:,:,i]=im_Cm[round(max(shift[0,:])):, round(max(shift[1,:])):,i]
 return data_aligned

In [ ]:
import numpy as np
def norm(data, thresh):
    data_unfold=unfold(data)
    mask=np.zeros((data.shape[0]*data.shape[1]))
    mask2=np.zeros((data.shape[0]*data.shape[1]))
    data_mask=np.zeros((data.shape[0]*data.shape[1], data.shape[2]))
    data_mask2=np.zeros((data.shape[0]*data.shape[1], data.shape[2]))
    for j in range(data.shape[2]):
     for i in range(data_unfold.shape[0]):
        if np.mean(data_unfold[i,:])>thresh:
            
            mask[i]=1
            mask2[i]=0
            data_mask[i,j]=data_unfold[i,j]
            data_mask2[i,j]=0
        else:###pixels that dont belong to the background have 0 flux
            mask[i]=0
            mask2[i]=1
            data_mask[i,j]=0
            data_mask2[i,j]=data_unfold[i,j]
    io_av=np.zeros((data.shape[2]))
    for j in range(data.shape[2]):
     io_av[j]=sum(data_mask[:,j])/sum(mask)
    data_whitened=np.zeros((data.shape[0], data.shape[1], data.shape[2]))
    data_whitened_alt=np.zeros((data.shape[0], data.shape[1], data.shape[2]))
    for k in range(data.shape[2]):
        data_whitened[:,:,k]=-np.log(abs(data[:,:,k]/io_av[k]))
        #data_whitened_alt[:,:,k]=data[:,:,k]/io_av[k]
        #print(k, data[:,:,k], io_av[k])
        #data_whitened[:,:,k]=data[:,:,k]/(io_av[k]/sum(mask))
    data_whitened_unfold=unfold(data_whitened)
    return data_whitened, data_whitened_unfold,mask,io_av

In [ ]:
def normtey(data, thresh1, thresh2):
    data_unfold=unfold(data)
    mask=np.zeros((data.shape[0]*data.shape[1]))
    mask2=np.zeros((data.shape[0]*data.shape[1]))
    data_mask=np.zeros((data.shape[0]*data.shape[1], data.shape[2]))
    data_mask2=np.zeros((data.shape[0]*data.shape[1], data.shape[2]))
    for j in range(data.shape[2]):
     for i in range(data_unfold.shape[0]):
        if (np.mean(data_unfold[i,:])>thresh1) and (np.mean(data_unfold[i,:])<thresh2):
            
            mask[i]=1
            mask2[i]=0
            data_mask[i,j]=data_unfold[i,j]
            data_mask2[i,j]=0
        else:###pixels that dont belong to the background have 0 flux
            mask[i]=0
            mask2[i]=1
            data_mask[i,j]=0
            data_mask2[i,j]=data_unfold[i,j]
    io_av=np.zeros((data.shape[2]))
    for j in range(data.shape[2]):
     io_av[j]=sum(data_mask[:,j])
    data_whitened=np.zeros((data.shape[0], data.shape[1], data.shape[2]))
    for k in range(data.shape[2]):
        data_whitened[:,:,k]=data[:,:,k]/abs((io_av[k]/sum(mask)))
        #data_whitened[:,:,k]=data[:,:,k]/(io_av[k]/sum(mask))
    data_whitened_unfold=unfold(data_whitened)
    return data_whitened, data_whitened_unfold,mask,io_av

In [ ]:
def gif(data, folder):
 import imageio
 imageio.mimsave(folder+'image.gif', np.transpose(data,(2,0,1)), fps=1)

In [ ]:
def pngtogif(energy, intensity, folder, labelx, labely):
 for i in range (int(intensity.shape[0])):
        fig=plt.plot(energy[:i+1], intensity[:i+1])
        plt.xlim([min(energy), max(energy)])
        plt.minorticks_on()
        plt.xlabel(labelx)
        plt.ylabel(labely)
       # plt.ylim([min(intensity)-0.1, max(intensity)+0.1])
        plt.savefig(folder+str(i)+'.png')
 import imageio
 from PIL import Image
 images = []
 for i in range(0, int(energy.shape[0]), 1):    
  print(i)
  images.append(imageio.imread(folder+str(i)+'.png')) 
  imageio.mimsave(folder+'spectrum.gif', images)

In [ ]:
def seqgif(data, energy, folder, n1, n2, resy, resx):
 for i in range (n1, n2, 1):
        fig=plt.imshow(data[:,:,i], cmap='gray', extent=[0,data.shape[1]*resy,0,data.shape[0]*resx])
        plt.minorticks_on()
        plt.title('E='+str(energy[i])+'eV')
        plt.savefig(folder+str(i)+'.png')
        plt.xlabel('y(um)')
        plt.ylabel('x(um)')
 import imageio
 from PIL import Image
 images = []
 for i in range(n1, n2, 1):    
  #print(i)
  images.append(imageio.imread(folder+str(i)+'.png')) 
  imageio.mimsave(folder+'seq.gif', images, fps=1)


def clusteringopt(data,nb_clusters, excl, energy,d,cl):
    from sklearn.cluster import SpectralClustering
    import matplotlib.pyplot as plt
    import numpy as np
    import h5py
    import sklearn.decomposition
    from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
    #from fcmeans import FCM
    #import skfuzzy as fuzz
    from sklearn.manifold import TSNE
    import collections
    from numpy.ma import masked_array
    from sklearn import mixture
    data_unfold=data.reshape(data.shape[0]*data.shape[1],data.shape[2])
    energy=energy
    def uni(records_array):
        '''
        get all indices of each unique value in array
        '''
        idx_sort = np.argsort(records_array)
        sorted_records_array = records_array[idx_sort]
        vals, idx_start, count = np.unique(sorted_records_array, return_counts=True, return_index=True)
        res = np.split(idx_sort, idx_start[1:])
        return dict(zip(vals, res))

    
    nb_principle_comp = 6
    exclude_first_component = excl
    dim_reduction_method = d #'ICA'|'PCA'|'KernelPCA'|'NMF'|'TSNE'
    cluster_method = cl #'kmeans'|'aggl_cl'|'DBSCAN'|'fcm'|'SpectralClustering'|'gmm'
    skip_dim_reduction = False
    normalize = False
    #############################Dimensionality reduction############################################################################

    if normalize:
     # every value gets divided by the mean of its spectrum
      data_unfold = (data_unfold/data_unfold.mean(0)[None,:])

        ### Dimension Reduction ###
    if not skip_dim_reduction:

            if dim_reduction_method == 'PCA':
                PCA=sklearn.decomposition.PCA
                pca=PCA(n_components=0.999) # specify amount of variance by float value, chooses nb of components accordingly 
                H=pca.fit_transform(data_unfold)
                H_refold=H.reshape(data.shape[0],data.shape[1],-1)
                W=pca.components_.T

            elif dim_reduction_method == 'ICA':
                ICA=sklearn.decomposition.FastICA
                ica=ICA(n_components=nb_principle_comp)
                H=ica.fit_transform(data_unfold)
                H_refold=H.reshape(data.shape[0],data.shape[1],-1)
                W=ica.components_.T

            elif dim_reduction_method == 'KernelPCA':
                # https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.KernelPCA.html
                PCA=sklearn.decomposition.KernelPCA
                #pca=PCA(n_components=nb_principle_comp, kernel='rbf')
                pca=PCA(n_components=nb_principle_comp, kernel='cosine', fit_inverse_transform = True)
                #H=pca.fit_transform(od)
                H=pca.fit_transform(data_unfold)
                pca.H_transformed_fit_
                H_refold=H.reshape(data.shape[0],data.shape[1],-1)
                W=pca.components_.T
            elif dim_reduction_method == 'NMF':
                NMF=sklearn.decomposition.NMF
                nmf=NMF(n_components=nb_principle_comp, max_iter=4000,  tol=1e-7)
                #tol=1e-4 default,init == ‘nndsvdar’ or ‘random’ or None
                data_unfold += abs(data_unfold.min()) # add minimum to every value to avoid negative values
                H=nmf.fit_transform(data_unfold)
                H_refold=H.reshape(data.shape[0],data.shape[1],-1)
                W=nmf.components_.T
            elif dim_reduction_method == 'TSNE':
               #TSNE=sklearn.manifold.TSNE
               #methods["t-SNE"] = manifold.TSNE(n_components=2, init="pca", random_state=0)
               #tsne= TSNE(n_components=2, learning_rate='auto', init='random')
               #H=tsne.fit_transform(od)
                H = TSNE(n_components=nb_principle_comp , init='random', random_state=0).fit_transform(data_unfold)
                H_refold=H.reshape(data.shape[0],data.shape[1],-1)
               

    ###########################Clustering###########################################
    if cluster_method=='kmeans':
            model = KMeans(n_clusters=nb_clusters, max_iter=5000,tol=0.0000001)
    elif cluster_method=='aggl_cl':
            model = KMeans(n_clusters=nb_clusters)
            model = AgglomerativeClustering(n_clusters=nb_clusters, affinity='cosine', linkage='average')
    elif cluster_method=='DBSCAN':
            model = DBSCAN()
    #elif cluster_method=='fcm':
            #model = FCM(n_clusters=nb_clusters) # we use two cluster as an example
    elif cluster_method=='gmm':
            model = mixture.GaussianMixture(n_components=nb_clusters, covariance_type='full', random_state=42)
    elif cluster_method=='SpectralClustering':
         model=SpectralClustering(n_clusters=nb_clusters, eigen_solver="arpack", affinity="nearest_neighbors")

    if not skip_dim_reduction:
            clusters = model.fit_predict(H[:,1:nb_principle_comp+1]) if exclude_first_component else model.fit_predict(H[:,:nb_principle_comp])
            clusters_img = clusters.reshape(data.shape[0],data.shape[1]) 

    else:
            clusters = model.fit_predict(data_unfold)
            clusters_img = clusters.reshape(data.shape[0],data.shape[1])
    #######kmeans seems pretty robust with noise and so I will not reduce the dimensionality of my data with PCA############
    ############################indexing#############################
#plt.figure(1)  
#plt.imshow(data.mean(2), cmap='gray')
#print('Original Image (mean)')
#plt.show()
    cluster_per_pixel = clusters_img
    cluster_idx = uni(cluster_per_pixel.reshape(-1))

        
    cluster_plot = plt.imshow(cluster_per_pixel, cmap='jet')
    #plt.imshow(cluster_per_pixel, 'jet')
    scale(cluster_per_pixel,resox,resoy, 'jet')
    print('Clustered Image')
    used_colors = cluster_plot.cmap(cluster_plot.norm(np.unique(cluster_per_pixel)))
    #plt.show()
    plt.figure(2) 
    for i in cluster_idx:

        idx = cluster_idx[i]
        #print('Overall Mean of cluster {}:'.format(i), od_fold.mean(2).reshape(-1)[idx].mean())
        #plt.plot(energy,od_fold.reshape(-1,data.shape[2])[idx].mean(0), c=used_colors[i], label='Cluster'+str(i))
        plt.plot(energy,data.reshape(-1,data.shape[2])[idx].mean(0)/max(data.reshape(-1,data.shape[2])[idx].mean(0)), c=used_colors[i], label='Cluster'+str(i))#/max(data.reshape(-1,data.shape[2])[idx].mean(0)), c=used_colors[i], label='Cluster'+str(i))
        #divide by /od_fold.reshape(-1,data.shape[2])[idx].mean(0)[62] to normalize
        plt.xlabel('Incident Photon Energy (eV)')
        #plt.ylabel('OD per pixel/ ODmax')
        plt.ylabel('Normalized to the maximum value')
        #plt.ylabel('Avergage Trans signal')
        plt.minorticks_on()   
       # plt.title('Normalized at last peak')
    print('Mean Spectrum of each cluster')
    plt.legend()
    plt.show()
        
    return  cluster_per_pixel,cluster_idx

In [ ]:
def clusteringbio(data,nb_clusters, excl, energy,d,cl):
    from sklearn.cluster import SpectralClustering
    import matplotlib.pyplot as plt
    import numpy as np
    import h5py
    import sklearn.decomposition
    from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
    #from fcmeans import FCM
    #import skfuzzy as fuzz
    from sklearn.manifold import TSNE
    import collections
    from numpy.ma import masked_array
    from sklearn import mixture
    data_unfold=data.reshape(data.shape[0]*data.shape[1],data.shape[2])
    energy=energy
    def uni(records_array):
        '''
        get all indices of each unique value in array
        '''
        idx_sort = np.argsort(records_array)
        sorted_records_array = records_array[idx_sort]
        vals, idx_start, count = np.unique(sorted_records_array, return_counts=True, return_index=True)
        res = np.split(idx_sort, idx_start[1:])
        return dict(zip(vals, res))

    
    nb_principle_comp = 6
    exclude_first_component = excl
    dim_reduction_method = d #'ICA'|'PCA'|'KernelPCA'|'NMF'|'TSNE'
    cluster_method = cl #'kmeans'|'aggl_cl'|'DBSCAN'|'fcm'|'SpectralClustering'|'gmm'
    skip_dim_reduction = False
    normalize = False
    #############################Dimensionality reduction############################################################################

    if normalize:
     # every value gets divided by the mean of its spectrum
      data_unfold = (data_unfold/data_unfold.mean(0)[None,:])

        ### Dimension Reduction ###
    if not skip_dim_reduction:

            if dim_reduction_method == 'PCA':
                PCA=sklearn.decomposition.PCA
                pca=PCA(n_components=0.999) # specify amount of variance by float value, chooses nb of components accordingly 
                H=pca.fit_transform(data_unfold)
                H_refold=H.reshape(data.shape[0],data.shape[1],-1)
                W=pca.components_.T

            elif dim_reduction_method == 'ICA':
                ICA=sklearn.decomposition.FastICA
                ica=ICA(n_components=nb_principle_comp)
                H=ica.fit_transform(data_unfold)
                H_refold=H.reshape(data.shape[0],data.shape[1],-1)
                W=ica.components_.T

            elif dim_reduction_method == 'KernelPCA':
                # https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.KernelPCA.html
                PCA=sklearn.decomposition.KernelPCA
                #pca=PCA(n_components=nb_principle_comp, kernel='rbf')
                pca=PCA(n_components=nb_principle_comp, kernel='cosine', fit_inverse_transform = True)
                #H=pca.fit_transform(od)
                H=pca.fit_transform(data_unfold)
                pca.H_transformed_fit_
                H_refold=H.reshape(data.shape[0],data.shape[1],-1)
                W=pca.components_.T
            elif dim_reduction_method == 'NMF':
                NMF=sklearn.decomposition.NMF
                nmf=NMF(n_components=nb_principle_comp, max_iter=4000,  tol=1e-7)
                #tol=1e-4 default,init == ‘nndsvdar’ or ‘random’ or None
                data_unfold += abs(data_unfold.min()) # add minimum to every value to avoid negative values
                H=nmf.fit_transform(data_unfold)
                H_refold=H.reshape(data.shape[0],data.shape[1],-1)
                W=nmf.components_.T
            elif dim_reduction_method == 'TSNE':
               #TSNE=sklearn.manifold.TSNE
               #methods["t-SNE"] = manifold.TSNE(n_components=2, init="pca", random_state=0)
               #tsne= TSNE(n_components=2, learning_rate='auto', init='random')
               #H=tsne.fit_transform(od)
                H = TSNE(n_components=nb_principle_comp , init='random', random_state=0).fit_transform(data_unfold)
                H_refold=H.reshape(data.shape[0],data.shape[1],-1)
               

    ###########################Clustering###########################################
    if cluster_method=='kmeans':
            model = KMeans(n_clusters=nb_clusters, max_iter=5000,tol=0.0000001)
    elif cluster_method=='aggl_cl':
            model = KMeans(n_clusters=nb_clusters)
            model = AgglomerativeClustering(n_clusters=nb_clusters, affinity='cosine', linkage='average')
    elif cluster_method=='DBSCAN':
            model = DBSCAN()
    #elif cluster_method=='fcm':
            #model = FCM(n_clusters=nb_clusters) # we use two cluster as an example
    elif cluster_method=='gmm':
            model = mixture.GaussianMixture(n_components=nb_clusters, covariance_type='full', random_state=42)
    elif cluster_method=='SpectralClustering':
         model=SpectralClustering(n_clusters=nb_clusters, eigen_solver="arpack", affinity="nearest_neighbors")

    if not skip_dim_reduction:
            clusters = model.fit_predict(H[:,1:nb_principle_comp+1]) if exclude_first_component else model.fit_predict(H[:,:nb_principle_comp])
            clusters_img = clusters.reshape(data.shape[0],data.shape[1]) 

    else:
            clusters = model.fit_predict(data_unfold)
            clusters_img = clusters.reshape(data.shape[0],data.shape[1])
    #######kmeans seems pretty robust with noise and so I will not reduce the dimensionality of my data with PCA############
    ############################indexing#############################
#plt.figure(1)  
#plt.imshow(data.mean(2), cmap='gray')
#print('Original Image (mean)')
#plt.show()
    cluster_per_pixel = clusters_img
    cluster_idx = uni(cluster_per_pixel.reshape(-1))

        
    cluster_plot = plt.imshow(cluster_per_pixel, cmap='jet')
    #plt.imshow(cluster_per_pixel, 'jet')
    plt.imshow(cluster_per_pixel, 'jet')
    print('Clustered Image')
    used_colors = cluster_plot.cmap(cluster_plot.norm(np.unique(cluster_per_pixel)))
    #plt.show()
    plt.figure(2) 
    for i in cluster_idx:

        idx = cluster_idx[i]
        #print('Overall Mean of cluster {}:'.format(i), od_fold.mean(2).reshape(-1)[idx].mean())
        #plt.plot(energy,od_fold.reshape(-1,data.shape[2])[idx].mean(0), c=used_colors[i], label='Cluster'+str(i))
        plt.plot(energy,data.reshape(-1,data.shape[2])[idx].mean(0)/max(data.reshape(-1,data.shape[2])[idx].mean(0)), c=used_colors[i], label='Cluster'+str(i))#/max(data.reshape(-1,data.shape[2])[idx].mean(0)), c=used_colors[i], label='Cluster'+str(i))
        #divide by /od_fold.reshape(-1,data.shape[2])[idx].mean(0)[62] to normalize
        plt.xlabel('Incident Photon Energy (eV)')
        #plt.ylabel('OD per pixel/ ODmax')
        plt.ylabel('Normalized to the maximum value')
        #plt.ylabel('Avergage Trans signal')
        plt.minorticks_on()   
       # plt.title('Normalized at last peak')
    print('Mean Spectrum of each cluster')
    plt.legend()
    plt.show()
        
    return  cluster_per_pixel,cluster_idx

In [ ]:
def clustering(data,nb_clusters, excl, energy):
    from sklearn.cluster import SpectralClustering
    import matplotlib.pyplot as plt
    import numpy as np
    import h5py
    import sklearn.decomposition
    from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
    #from fcmeans import FCM
    #import skfuzzy as fuzz
    from sklearn.manifold import TSNE
    import collections
    from numpy.ma import masked_array
    from sklearn import mixture
    data_unfold=data.reshape(data.shape[0]*data.shape[1],data.shape[2])
    energy=energy
    def uni(records_array):
        '''
        get all indices of each unique value in array
        '''
        idx_sort = np.argsort(records_array)
        sorted_records_array = records_array[idx_sort]
        vals, idx_start, count = np.unique(sorted_records_array, return_counts=True, return_index=True)
        res = np.split(idx_sort, idx_start[1:])
        return dict(zip(vals, res))

    
    nb_principle_comp = 6
    exclude_first_component = excl
    dim_reduction_method = 'PCA' #'ICA'|'PCA'|'KernelPCA'|'NMF'|'TSNE'
    cluster_method = 'kmeans' #'kmeans'|'aggl_cl'|'DBSCAN'|'fcm'|'SpectralClustering'|'gmm'
    skip_dim_reduction = False
    normalize = False
    #############################Dimensionality reduction############################################################################

    if normalize:
     # every value gets divided by the mean of its spectrum
      data_unfold = (data_unfold/data_unfold.mean(0)[None,:])

        ### Dimension Reduction ###
    if not skip_dim_reduction:

            if dim_reduction_method == 'PCA':
                PCA=sklearn.decomposition.PCA
                pca=PCA(n_components=0.999) # specify amount of variance by float value, chooses nb of components accordingly 
                H=pca.fit_transform(data_unfold)
                H_refold=H.reshape(data.shape[0],data.shape[1],-1)
                W=pca.components_.T

            elif dim_reduction_method == 'ICA':
                ICA=sklearn.decomposition.FastICA
                ica=ICA(n_components=nb_principle_comp)
                H=ica.fit_transform(data_unfold)
                H_refold=H.reshape(data.shape[0],data.shape[1],-1)
                W=ica.components_.T

            elif dim_reduction_method == 'KernelPCA':
                # https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.KernelPCA.html
                PCA=sklearn.decomposition.KernelPCA
                #pca=PCA(n_components=nb_principle_comp, kernel='rbf')
                pca=PCA(n_components=nb_principle_comp, kernel='cosine', fit_inverse_transform = True)
                #H=pca.fit_transform(od)
                H=pca.fit_transform(data_unfold)
                pca.H_transformed_fit_
                H_refold=H.reshape(data.shape[0],data.shape[1],-1)
                W=pca.components_.T
            elif dim_reduction_method == 'NMF':
                NMF=sklearn.decomposition.NMF
                nmf=NMF(n_components=nb_principle_comp, max_iter=4000,  tol=1e-7)
                #tol=1e-4 default,init == ‘nndsvdar’ or ‘random’ or None
                data_unfold += abs(data_unfold.min()) # add minimum to every value to avoid negative values
                H=nmf.fit_transform(data_unfold)
                H_refold=H.reshape(data.shape[0],data.shape[1],-1)
                W=nmf.components_.T
            elif dim_reduction_method == 'TSNE':
               #TSNE=sklearn.manifold.TSNE
               #methods["t-SNE"] = manifold.TSNE(n_components=2, init="pca", random_state=0)
               #tsne= TSNE(n_components=2, learning_rate='auto', init='random')
               #H=tsne.fit_transform(od)
               H = TSNE(n_components=nb_principle_comp , init='random', random_state=0).fit_transform(data_unfold)
               H_refold=H.reshape(data.shape[0],data.shape[1],-1)
               

    ###########################Clustering###########################################
    if cluster_method=='kmeans':
            model = KMeans(n_clusters=nb_clusters, max_iter=5000,tol=0.0000001)
    elif cluster_method=='aggl_cl':
            model = KMeans(n_clusters=nb_clusters)
            model = AgglomerativeClustering(n_clusters=nb_clusters, affinity='cosine', linkage='average')
    elif cluster_method=='DBSCAN':
            model = DBSCAN()
    #elif cluster_method=='fcm':
            #model = FCM(n_clusters=nb_clusters) # we use two cluster as an example
    elif cluster_method=='gmm':
            model = mixture.GaussianMixture(n_components=nb_clusters, covariance_type='full', random_state=42)
    elif cluster_method=='SpectralClustering':
         model=SpectralClustering(n_clusters=nb_clusters, eigen_solver="arpack", affinity="nearest_neighbors")

    if not skip_dim_reduction:
            clusters = model.fit_predict(H[:,1:nb_principle_comp+1]) if exclude_first_component else model.fit_predict(H[:,:nb_principle_comp])

            clusters_img = clusters.reshape(data.shape[0],data.shape[1]) 

    else:
            clusters = model.fit_predict(data_unfold)
            clusters_img = clusters.reshape(data.shape[0],data.shape[1])
    #######kmeans seems pretty robust with noise and so I will not reduce the dimensionality of my data with PCA############
    ############################indexing#############################
    cluster_per_pixel = clusters_img
    cluster_idx = uni(cluster_per_pixel.reshape(-1))
    #plt.figure(1)  
    #plt.imshow(data.mean(2), cmap='gray')
    #print('Original Image (mean)')
    #plt.show()
        
    cluster_plot = plt.imshow(cluster_per_pixel, cmap='jet')
    #plt.imshow(cluster_per_pixel, 'jet')
    plt.imshow(cluster_per_pixel, 'jet')
    print('Clustered Image')
    used_colors = cluster_plot.cmap(cluster_plot.norm(np.unique(cluster_per_pixel)))
    #plt.show()
    plt.figure(2) 
    for i in cluster_idx:

        idx = cluster_idx[i]
        #print('Overall Mean of cluster {}:'.format(i), od_fold.mean(2).reshape(-1)[idx].mean())
        #plt.plot(energy,od_fold.reshape(-1,data.shape[2])[idx].mean(0), c=used_colors[i], label='Cluster'+str(i))
        plt.plot(energy,data.reshape(-1,data.shape[2])[idx].mean(0)/max(data.reshape(-1,data.shape[2])[idx].mean(0)), c=used_colors[i], label='Cluster'+str(i))#/max(data.reshape(-1,data.shape[2])[idx].mean(0)), c=used_colors[i], label='Cluster'+str(i))
        #divide by /od_fold.reshape(-1,data.shape[2])[idx].mean(0)[62] to normalize
        plt.xlabel('Incident Photon Energy (eV)')
        #plt.ylabel('OD per pixel/ ODmax')
        plt.ylabel('Normalized to the maximum value')
        #plt.ylabel('Avergage Trans signal')
        plt.minorticks_on()   
       # plt.title('Normalized at last peak')
    print('Mean Spectrum of each cluster')
    plt.show()
        
    #return  cluster_per_pixel

In [ ]:
def odclustering(f,nb_clusters, excl):
 data=f['exchange/data'][...]
 data=np.transpose(data,(1,0,2))
 data=np.flipud(data)
 energy=f['exchange/energy'][...]
 x,y=f['exchange/x'][...], f['exchange/y'][...]
 od=f['spectromicroscopy/optical_density'][...]
 od_fold=od.reshape(data.shape[0],data.shape[1],data.shape[2])
 od_fold=np.flipud(od_fold)
 plt.figure(0) 
 plt.imshow(np.sum(od_fold,axis=2))
###################################get all indices of each unique value in array#####################################################################################
 def uni(records_array):
    '''
    get all indices of each unique value in array
    '''
    idx_sort = np.argsort(records_array)
    sorted_records_array = records_array[idx_sort]
    vals, idx_start, count = np.unique(sorted_records_array, return_counts=True, return_index=True)
    res = np.split(idx_sort, idx_start[1:])
    return dict(zip(vals, res))


 nb_principle_comp = 3
 exclude_first_component = excl
 dim_reduction_method = 'PCA' #'ICA'|'PCA'|'KernelPCA'|'NMF'|'TSNE'
 cluster_method = 'kmeans' #'kmeans'|'aggl_cl'|'DBSCAN'|'fcm'|'SpectralClustering'|'gmm'
 skip_dim_reduction = False
 normalize = False
 plotpca=False
#############################Dimensionality reduction############################################################################

 if normalize:
 # every value gets divided by the mean of its spectrum
  od = (od/od.mean(0)[None,:])

    ### Dimension Reduction ###
 if not skip_dim_reduction:

        if dim_reduction_method == 'PCA':
            PCA=sklearn.decomposition.PCA
            pca=PCA(n_components=0.999) # specify amount of variance by float value, chooses nb of components accordingly 
            H=pca.fit_transform(od)
            H_refold=H.reshape(data.shape[0],data.shape[1],-1)
            W=pca.components_.T

        elif dim_reduction_method == 'ICA':
            ICA=sklearn.decomposition.FastICA
            ica=ICA(n_components=nb_principle_comp)
            H=ica.fit_transform(od)
            H_refold=H.reshape(data.shape[0],data.shape[1],-1)
            W=ica.components_.T

        elif dim_reduction_method == 'KernelPCA':
            # https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.KernelPCA.html
            PCA=sklearn.decomposition.KernelPCA
            #pca=PCA(n_components=nb_principle_comp, kernel='rbf')
            pca=PCA(n_components=nb_principle_comp, kernel='cosine', fit_inverse_transform = True)
            #H=pca.fit_transform(od)
            H=pca.fit_transform(od)
            pca.H_transformed_fit_
            H_refold=H.reshape(data.shape[0],data.shape[1],-1)
            W=pca.components_.T
        elif dim_reduction_method == 'NMF':
            NMF=sklearn.decomposition.NMF
            nmf=NMF(n_components=nb_principle_comp, max_iter=4000,  tol=1e-7)
            #tol=1e-4 default,init == ‘nndsvdar’ or ‘random’ or None
            od += abs(od.min()) # add minimum to every value to avoid negative values
            H=nmf.fit_transform(od)
            H_refold=H.reshape(data.shape[0],data.shape[1],-1)
            W=nmf.components_.T
        elif dim_reduction_method == 'TSNE':
           #TSNE=sklearn.manifold.TSNE
           #methods["t-SNE"] = manifold.TSNE(n_components=2, init="pca", random_state=0)
           #tsne= TSNE(n_components=2, learning_rate='auto', init='random')
           #H=tsne.fit_transform(od)
           H = TSNE(n_components=nb_principle_comp , init='random', random_state=0).fit_transform(od)
           H_refold=H.reshape(data.shape[0],data.shape[1],-1)         
 ###########################Clustering###########################################
 if cluster_method=='kmeans':
        model = KMeans(n_clusters=nb_clusters, max_iter=5000,tol=0.0000001)
 elif cluster_method=='aggl_cl':
        model = KMeans(n_clusters=nb_clusters)
        model = AgglomerativeClustering(n_clusters=nb_clusters, affinity='cosine', linkage='average')
 elif cluster_method=='DBSCAN':
        model = DBSCAN()
 #elif cluster_method=='fcm':
        #model = FCM(n_clusters=nb_clusters) # we use two cluster as an example
 elif cluster_method=='gmm':
        model = mixture.GaussianMixture(n_components=nb_clusters, covariance_type='full', random_state=42)
 elif cluster_method=='SpectralClustering':
     model=SpectralClustering(n_clusters=nb_clusters, eigen_solver="arpack", affinity="nearest_neighbors")

 if not skip_dim_reduction:
        clusters = model.fit_predict(H[:,1:nb_principle_comp+1]) if exclude_first_component else model.fit_predict(H[:,:nb_principle_comp])
        #clusters = model.cmeans(H[:,1:nb_principle_comp+1], c=nb_clusters, m=2 , error=1e-4, maxiter'=1000)
       # clusters = model.cmeans_predict(H[:,1:nb_principle_comp+1], ncenters, 2, error=0.005, maxiter=1000, init=None)
                                        #'cntr_trained', 'm', 'error', 'maxiter') 
        ##fuzzy##
      #  q = model.fit(H[:,1:nb_principle_comp+1]) if exclude_first_component else model.fit(H[:,:nb_principle_comp])
       # clusters = model.predict(H[:,1:nb_principle_comp+1]) if exclude_first_component else model.predict(H[:,:nb_principle_comp])
        #clusters_img = clusters.reshape(data.shape[0],data.shape[1])
        #centers = model.centers
        ###########################
        ##gmm##
       # q = model.fit(H[:,1:nb_principle_comp+1]) if exclude_first_component else model.fit(H[:,:nb_principle_comp])
       # clusters = model.predict(H[:,1:nb_principle_comp+1]) if exclude_first_component else model.predict(H[:,:nb_principle_comp])
        clusters_img = clusters.reshape(data.shape[0],data.shape[1]) 
       # probs = model.predict_proba(H[:,1:nb_principle_comp+1]) if exclude_first_component else model.predict_proba(H[:,:nb_principle_comp])
        #probs_fold=probs.reshape(data.shape[0],data.shape[1],nb_clusters)
       # ###########################
 else:
        ##fuzzy##
       # clusters = model.fit(od)
        #centers = model.centers
        #labels = model.predict(od)
        #clusters_img
        ##gmm##
       # clusters = model.fit(od)
       # clusters_img = clusters.reshape(data.shape[0],data.shape[1])
       # probs = model.predict_proba(od)
       # probs_fold=probs.reshape(data.shape[0],data.shape[1],nb_clusters)
       # print(probs[:5].round(3))
        #########= labels.reshape(data.shape[0],data.shape[1])
        #########
        clusters = model.fit_predict(od)
        clusters_img = clusters.reshape(data.shape[0],data.shape[1])
#######kmeans seems pretty robust with noise and so I will not reduce the dimensionality of my data with PCA############
############################indexing#############################
 cluster_per_pixel = np.flipud(clusters_img)
 cluster_idx = uni(cluster_per_pixel.reshape(-1))
######################################################
######################Clustered image################################################
 
    
 cluster_plot = plt.imshow(cluster_per_pixel, cmap='jet')
 print('Clustered Image')
 used_colors = cluster_plot.cmap(cluster_plot.norm(np.unique(cluster_per_pixel)))
 plt.show()
######################Clustered spectra################################################
 return od_fold.reshape(-1,data.shape[2])[cluster_idx[0]].mean(0)

In [ ]:
def fineplot(x,y): 
 fig, ax2 = plt.subplots(1,1)
 plt.plot(x,y,color='gray',linewidth=3.0,label='Single-layered MXene')
 plt.rcParams['font.size'] = '18'
 plt.xlabel('Incident Photon Energy (eV)',weight="bold", fontsize=19)
 matplotlib.rcParams.update({'font.size': 19})
 #plt.xlim([450,475])
 #plt.ylim([0.03,0.47])
 plt.ylabel('Intensity (arb.units)',weight="bold", fontsize=19)
 plt.yticks([])
 plt.minorticks_on()   
 for axis in ['top','bottom','left','right']:
    ax2.spines[axis].set_linewidth(2)
 ax2.tick_params(width=2)
 plt.show()

In [ ]:
def clusteringnew(data,nb_clusters, excl, energy,dim,cl):
    from sklearn.cluster import SpectralClustering
    import matplotlib.pyplot as plt
    import numpy as np
    import h5py
    import sklearn.decomposition
    from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
    #from fcmeans import FCM
    #import skfuzzy as fuzz
    from sklearn.manifold import TSNE
    import collections
    from numpy.ma import masked_array
    from sklearn import mixture
    data_unfold=data.reshape(data.shape[0]*data.shape[1],data.shape[2])
    energy=energy
    def uni(records_array):
        '''
        get all indices of each unique value in array
        '''
        idx_sort = np.argsort(records_array)
        sorted_records_array = records_array[idx_sort]
        vals, idx_start, count = np.unique(sorted_records_array, return_counts=True, return_index=True)
        res = np.split(idx_sort, idx_start[1:])
        return dict(zip(vals, res))

    
    nb_principle_comp = 6
    exclude_first_component = excl
    dim_reduction_method = dim #'ICA'|'PCA'|'KernelPCA'|'NMF'|'TSNE'
    cluster_method = cl #'kmeans'|'aggl_cl'|'DBSCAN'|'fcm'|'SpectralClustering'|'gmm'
    skip_dim_reduction = False
    normalize = False
    #############################Dimensionality reduction############################################################################

    if normalize:
     # every value gets divided by the mean of its spectrum
      data_unfold = (data_unfold/data_unfold.mean(0)[None,:])

        ### Dimension Reduction ###
    if not skip_dim_reduction:

            if dim_reduction_method == 'PCA':
                PCA=sklearn.decomposition.PCA
                pca=PCA(n_components=0.999) # specify amount of variance by float value, chooses nb of components accordingly 
                H=pca.fit_transform(data_unfold)
                H_refold=H.reshape(data.shape[0],data.shape[1],-1)
                W=pca.components_.T

            elif dim_reduction_method == 'ICA':
                ICA=sklearn.decomposition.FastICA
                ica=ICA(n_components=nb_principle_comp)
                H=ica.fit_transform(data_unfold)
                H_refold=H.reshape(data.shape[0],data.shape[1],-1)
                W=ica.components_.T

            elif dim_reduction_method == 'KernelPCA':
                # https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.KernelPCA.html
                PCA=sklearn.decomposition.KernelPCA
                #pca=PCA(n_components=nb_principle_comp, kernel='rbf')
                pca=PCA(n_components=nb_principle_comp, kernel='cosine', fit_inverse_transform = True)
                #H=pca.fit_transform(od)
                H=pca.fit_transform(data_unfold)
                pca.H_transformed_fit_
                H_refold=H.reshape(data.shape[0],data.shape[1],-1)
                W=pca.components_.T
            elif dim_reduction_method == 'NMF':
                NMF=sklearn.decomposition.NMF
                nmf=NMF(n_components=nb_principle_comp, max_iter=4000,  tol=1e-7)
                #tol=1e-4 default,init == ‘nndsvdar’ or ‘random’ or None
                data_unfold += abs(data_unfold.min()) # add minimum to every value to avoid negative values
                H=nmf.fit_transform(data_unfold)
                H_refold=H.reshape(data.shape[0],data.shape[1],-1)
                W=nmf.components_.T
            elif dim_reduction_method == 'TSNE':
               #TSNE=sklearn.manifold.TSNE
               #methods["t-SNE"] = manifold.TSNE(n_components=2, init="pca", random_state=0)
               #tsne= TSNE(n_components=2, learning_rate='auto', init='random')
               #H=tsne.fit_transform(od)
               H = TSNE(n_components=nb_principle_comp , init='random', random_state=0).fit_transform(data_unfold)
               H_refold=H.reshape(data.shape[0],data.shape[1],-1)
               

    ###########################Clustering###########################################
    if cluster_method=='kmeans':
            model = KMeans(n_clusters=nb_clusters, max_iter=5000,tol=0.0000001)
    elif cluster_method=='aggl_cl':
            model = KMeans(n_clusters=nb_clusters)
            model = AgglomerativeClustering(n_clusters=nb_clusters, affinity='cosine', linkage='average')
    elif cluster_method=='DBSCAN':
            model = DBSCAN()
    #elif cluster_method=='fcm':
            #model = FCM(n_clusters=nb_clusters) # we use two cluster as an example
    elif cluster_method=='gmm':
            model = mixture.GaussianMixture(n_components=nb_clusters, covariance_type='full', random_state=42)
    elif cluster_method=='SpectralClustering':
         model=SpectralClustering(n_clusters=nb_clusters, eigen_solver="arpack", affinity="nearest_neighbors")

    if not skip_dim_reduction:
            clusters = model.fit_predict(H[:,1:nb_principle_comp+1]) if exclude_first_component else model.fit_predict(H[:,:nb_principle_comp])

            clusters_img = clusters.reshape(data.shape[0],data.shape[1]) 

    else:
            clusters = model.fit_predict(data_unfold)
            clusters_img = clusters.reshape(data.shape[0],data.shape[1])
    #######kmeans seems pretty robust with noise and so I will not reduce the dimensionality of my data with PCA############
    ############################indexing#############################
    cluster_per_pixel = clusters_img
    cluster_idx = uni(cluster_per_pixel.reshape(-1))
    #plt.figure(1)  
    #plt.imshow(data.mean(2), cmap='gray')
    #print('Original Image (mean)')
    #plt.show()
        
    cluster_plot = plt.imshow(cluster_per_pixel, cmap='jet')
    #plt.imshow(cluster_per_pixel, 'jet')
    scale(cluster_per_pixel,resox,resoy, 'jet')
    print('Clustered Image')
    used_colors = cluster_plot.cmap(cluster_plot.norm(np.unique(cluster_per_pixel)))
    #plt.show()
    plt.figure(2) 
    for i in cluster_idx:

        idx = cluster_idx[i]
        #print('Overall Mean of cluster {}:'.format(i), od_fold.mean(2).reshape(-1)[idx].mean())
        #plt.plot(energy,od_fold.reshape(-1,data.shape[2])[idx].mean(0), c=used_colors[i], label='Cluster'+str(i))
        plt.plot(energy,data.reshape(-1,data.shape[2])[idx].mean(0)/max(data.reshape(-1,data.shape[2])[idx].mean(0)), c=used_colors[i], label='Cluster'+str(i))#/max(data.reshape(-1,data.shape[2])[idx].mean(0)), c=used_colors[i], label='Cluster'+str(i))
        #divide by /od_fold.reshape(-1,data.shape[2])[idx].mean(0)[62] to normalize
        plt.xlabel('Incident Photon Energy (eV)')
        #plt.ylabel('OD per pixel/ ODmax')
        plt.ylabel('Normalized to the maximum value')
        #plt.ylabel('Avergage Trans signal')
        plt.minorticks_on()  
        plt.legend()
       # plt.title('Normalized at last peak')
    print('Mean Spectrum of each cluster')
    plt.show()
    
        
    return  cluster_per_pixel,cluster_idx